In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import re
from re import sub
from decimal import Decimal
import random
import pandas as pd
from urllib.request import urlopen
import dateutil.parser
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 12]


from fake_useragent import UserAgent
import os



chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

import json



In [2]:
zipcodes=pd.read_csv('bayarea_zipcodes.csv')


In [58]:
zipcodes.ZIP
# len(zipcodes.ZIP)
# 187*0.8*500

94133

In [66]:
with open('house_url.json', 'r') as f:
     house_url=json.load(f)
len(house_url)

46246

In [62]:
#house_url=[]
#setting up fake user agent
ua = UserAgent()
user_agent = {'User-agent': ua.random}

#setting user agent names
opts=Options()
opts.add_argument('user-agent=user_agent')
driver = webdriver.Chrome(chromedriver, chrome_options=opts)
#only the recently sold homes
driver.get(url="https://www.zillow.com/ca/sold/")   

for zipcode_i in zipcodes.ZIP:

    sleeptime=random.uniform(5,7)
    time.sleep(sleeptime)
    #enter zipcode
    #I will only have 500 homes for each zipcode allocated around all places because
    #zillow requires you to zoom in to different map areas for more houses
    zipcode = driver.find_element_by_xpath("//input[@id='citystatezip']")
    zipcode.send_keys(Keys.BACKSPACE)
    zipcode.send_keys(Keys.BACKSPACE)
    zipcode.send_keys(Keys.BACKSPACE)
    zipcode.send_keys(Keys.BACKSPACE)
    zipcode.send_keys(Keys.BACKSPACE)
    zipcode.send_keys(str(zipcode_i))
    zipcode.send_keys(Keys.RETURN)
    sleeptime=random.uniform(4,6)
    time.sleep(sleeptime)
    
    #click on next page
    try:
        driver.find_element_by_class_name('zsg-pagination-next').click()
        sleeptime=random.uniform(3,5)
        time.sleep(sleeptime)
        #second page url
        url2=driver.current_url
        print(url2)
        ##### get the url to feed into beautifulsoup
        url_list=[]
        url1=url2[:-4]+str(1)+url2[-3:]
        url_list.append(url1)
        url_list.append(url2)
        #for i in range (page 3 to total page+1)
        for i in range(3,21):
            url_list.append(url2[:-4]+str(i)+url2[-3:])


        for j in url_list:
            #random user-agent
            ua = UserAgent()
            user_agent = {'User-agent': ua.random}
            #start using beautiful soup
            response = requests.get(j, headers=user_agent)
            sleeptime=random.uniform(3,5)
            time.sleep(sleeptime)
            print(response.status_code)
            soup=BeautifulSoup(response.text, "lxml")
            #url for individual houses
            for i in soup.find_all("a", {"class" :"zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link"}):
                house_url.append('zillow.com'+i['href'])
    except NoSuchElementException:
        print('no houses sold in this area')
        sleeptime=random.uniform(2,4)
        time.sleep(sleeptime)
        continue



https://www.zillow.com/homes/recently_sold/Oakland-CA-94606/97808_rid/globalrelevanceex_sort/37.821243,-122.189341,37.759112,-122.302638_rect/12_zm/2_p/
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
https://www.zillow.com/homes/recently_sold/San-Francisco-CA-94111/97566_rid/globalrelevanceex_sort/37.814361,-122.371345,37.783299,-122.427993_rect/13_zm/2_p/
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
https://www.zillow.com/homes/recently_sold/Oakland-CA-94619/97820_rid/globalrelevanceex_sort/37.848154,-122.039052,37.723885,-122.265645_rect/11_zm/2_p/
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
https://www.zillow.com/homes/recently_sold/San-Francisco-CA-94121/97575_rid/globalrelevanceex_sort/37.792727,-122.464815,37.761656,-122.521463_rect/13_zm/2_p/
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
https://www.zillow.com/homes/recently_sold/San-Francisco

In [64]:
with open('house_url.json', 'w') as f:
     json.dump(house_url, f)

## Open individual houses to get the characteristics using the house_url csv file stored from selenium

In [101]:
#open individual house's url using beautiful soup and extract all housing info

#setting up dataframe as a list
df_columns=['street address','city','state','zip',
            'bed','bath','sqft','type','year-built',
            'heating','cooling','parking','lot',
           'school1','school2','school3','grade1','grade2','grade3','rate1','rate2','rate3',
           'sold-price','sold-date','url']
data=[]

for j in house_url:
    #using random user-agent
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}
    #feed into beautiful soup
    response = requests.get(url='https://www.'+j, headers=user_agent)
    sleeptime=random.uniform(3,5)
    time.sleep(sleeptime)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    #getting individual house characteristics
    housedata=[]
    #street address
    for i in soup.find_all("header", {"class" : "zsg-content-header addr"}):
        housedata.append(i.text.split(',')[0])
    #city, state, zip
    for i in soup.find_all("span", {"class" : "zsg-h2 addr_city"}):
        city=i.text.split(',')
        housedata.append(city[0])
        housedata.append(city[1].split(' ')[1])
        housedata.append(city[1].split(' ')[2])
    #bed, bath, sqft
    for i in soup.find_all("span", {"class" : "addr_bbs"}):
        housedata.append(i.text.split(' ')[0])
    #facts and features table: type, year built, heating, cooling, parking, lotsize
    for i in soup.find_all("div", {"class" :"hdp-fact-ataglance-value"}):
        housedata.append(i.text.strip())
    #school name
    for i in soup.find_all("div", {"class" :'nearby-schools-name'}):
        housedata.append(i.text.strip())    
    #school grades
    for i in soup.find_all("div", {"class" :'nearby-schools-grades'}):
        housedata.append(i.text.strip())
    #school rating 
    for i in soup.find_all("div", {"class" :'nearby-schools-rating'}):
        housedata.append(i.text.strip().split(' ')[0])
    
    #prices
    
    price=[]
    for i in soup.find_all("div", {"class" : "home-summary-row"}):
        #housedata.append(i.text.strip())
        price.append(i.text.strip())

    try:
        sold_price=Decimal(sub(r'[^\d.]','',re.findall('[£$€]{1}[,0-9]{1,10}',price[0])[0]))
        sold_date=dateutil.parser.parse(price[1],fuzzy=True)
    except IndexError:
        print("for sale houses")
        continue
        
    housedata.append(sold_price)
    housedata.append(sold_date)

        
    housedata.append('https://www.'+j)

    #append list of individual house characteristics into data
    data.append(housedata)

    


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
for sale houses
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [102]:
#converting the data list into a dataframe
housedf=pd.DataFrame(data,columns=df_columns)

In [103]:
housedf

,street address,city,state,zip,bed,bath,sqft,type,year-built,heating,cooling,parking,lot,shool1,school2,school3,grade1,grade2,grade3,rate1,rate2,rate3,sold-price,sold-date,url
0,998 Daffodil Way,San Jose,CA,95117,3,2,"1,250",Single Family,1962,Forced air,None,2 spaces,"6,634 sqft",Easterbrook Discovery,Prospect High,George C. Payne Elementary,K-8,9-12,K-5,9,9,9,1090000,2017-07-10,https://www.zillow.com/homedetails/998-Daffodi...
1,519 Northlake Dr APT 2,San Jose,CA,95117,2,1,800,Apartment,No Data,No Data,No Data,"Carport, Off street",No Data,Lynhaven Elementary,Monroe Middle,Del Mar High,K-5,5-8,9-12,6,5,7,1630000,2017-06-30,https://www.zillow.com/homedetails/519-Northla...
2,4034 Payne Ave,San Jose,CA,95117,3,2,"1,248",Single Family,1958,Other,No Data,2 spaces,"6,098 sqft",George C. Payne Elementary,Prospect High,Easterbrook Discovery,K-5,9-12,K-8,9,9,9,1200006,2017-06-23,https://www.zillow.com/homedetails/4034-Payne-...
3,1255 Flora Ave,San Jose,CA,95117,5,2,"2,074",Multi Family,1960,No Data,No Data,"Garage - Attached, On street",No Data,Leroy Anderson Elementary,Prospect High,Easterbrook Discovery,K-5,9-12,K-8,3,9,9,1170000,2017-06-19,https://www.zillow.com/homedetails/1255-Flora-...
4,1359 Phelps Ave APT 8,San Jose,CA,95117,3,2,"1,107",Condo,1963,Other,No Data,1 space,$287/month,George C. Payne Elementary,Easterbrook Discovery,Boynton High,K-5,K-8,9-12,9,9,NR,605000,2017-06-15,https://www.zillow.com/homedetails/1359-Phelps...
5,3564 Parkland Ave,San Jose,CA,95117,5,3,"2,199",Single Family,1962,Forced air,Central,2 spaces,"6,098 sqft",George C. Payne Elementary,Prospect High,Campbell Middle,K-5,9-12,5-8,9,9,3,1450000,2017-06-14,https://www.zillow.com/homedetails/3564-Parkla...
6,556 Westridge Dr,San Jose,CA,95117,3,2,"1,291",Single Family,1955,Other,No Data,2 spaces,"6,054 sqft",Merritt Trace Elementary,Herbert Hoover Middle,Abraham Lincoln High,K-5,6-8,9-12,6,4,7,1200000,2017-06-09,https://www.zillow.com/homedetails/556-Westrid...
7,626 Coakley Dr,San Jose,CA,95117,3,2,"1,472",Single Family,1955,Other,No Data,2 spaces,"6,838 sqft",Lynhaven Elementary,Monroe Middle,Del Mar High,K-5,5-8,9-12,6,5,7,1058000,2017-06-09,https://www.zillow.com/homedetails/626-Coakley...
8,3128 Loma Verde Dr APT 118,San Jose,CA,95117,2,2,932,Condo,2007,Forced air,"Central, Other","Carport, Off street, On street",$285/month,Rosemary Elementary,Campbell Middle,Westmont High,K-4,5-8,9-12,4,3,9,605000,2017-06-09,https://www.zillow.com/homedetails/3128-Loma-V...
9,411 Northlake Dr APT 5,San Jose,CA,95117,3,3,"1,626",Condo,1985,Other,No Data,2 spaces,$458/month,Lynhaven Elementary,Monroe Middle,Del Mar High,K-5,5-8,9-12,6,5,7,848000,2017-06-07,https://www.zillow.com/homedetails/411-Northla...


In [104]:
#save data into a csv file
housedf.to_csv('data_95117.csv', index=False)
#studio will have 0 bedrooms